In [1]:
# install langchain and all other helper libraries
# chromadb is the database to store vector embedings
!pip3 install --upgrade pip
!pip3 install --quiet --upgrade langchain langchain-community langchain-openai chromadb
# pypdf reads pdf using python
# streamlit is the api to build webapplication using python
!pip3 install --quiet  --upgrade pypdf pandas streamlit python-dotenv

# 1. Import all necessary Packages and Modules

In [2]:
# import langchain modules
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

import os
import shutil
import tempfile
import streamlit as st
import pandas as pd
from dotenv import load_dotenv

/Users/prerakthakur/Documents/Learning/AI/RAG/RAG_from_pdf_project1/venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
# read all key value value pairs from hidden file .env and sets them as environment variables
load_dotenv()
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

# 2. Define and load llm

In [4]:
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0)

# test the model
llm.invoke
response = llm.invoke("what are the 7 wonders of the worlds?")
print(response.content)

The term "Seven Wonders of the World" can refer to different lists, but the two most commonly referenced are the **Seven Wonders of the Ancient World** and the **New Seven Wonders of the World**.

### Seven Wonders of the Ancient World:
1. **Great Pyramid of Giza** (Egypt) - The only surviving wonder, it was built as a tomb for the Pharaoh Khufu.
2. **Hanging Gardens of Babylon** (Iraq) - Said to be an extraordinary series of tiered gardens, though its existence is debated.
3. **Statue of Zeus at Olympia** (Greece) - A giant statue made by the sculptor Phidias, honoring the god Zeus.
4. **Temple of Artemis at Ephesus** (Turkey) - A large temple dedicated to the goddess Artemis, known for its grandeur.
5. **Mausoleum at Halicarnassus** (Turkey) - A tomb built for Mausolus, a satrap of the Persian Empire, known for its architectural beauty.
6. **Colossus of Rhodes** (Greece) - A giant statue that stood at the entrance of the harbor of Rhodes.
7. **Lighthouse of Alexandria (Pharos of Alex

# 3. Load the PDF file and split it in chunks

In [5]:
# load the pdf that you want RAG on
loader = PyPDFLoader("data/world_religions.pdf")

# load function returns a list of all the pages of pdf
document_pages = loader.load()

# split the document in chunks for easy access to relevant information and avoid providing the whole document in the context
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, length_function=len, separators=["\n\n", "\n", " "])
chunks = text_splitter.split_documents(document_pages)
print(len(chunks))

Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 23 0 (offset 0)
Ignoring wrong pointing object 29 0 (offset 0)
Ignoring wrong pointing object 36 0 (offset 0)
Ignoring wrong pointing object 39 0 (offset 0)
Ignoring wrong pointing object 41 0 (offset 0)
Ignoring wrong pointing object 50 0 (offset 0)
Ignoring wrong pointing object 67 0 (offset 0)
Ignoring wrong pointing object 285 0 (offset 0)
Ignoring wrong pointing object 288 0 (offset 0)
Ignoring wrong pointing object 295 0 (offset 0)
Ignoring wrong pointing object 306 0 (offset 0)
Ignoring wrong pointing object 311 0 (offset 0)
Ignoring wrong pointing object 324 0 (offset 0)
Ignoring wrong pointing object 330 0 (offset 0)
Ignoring wrong pointing object 332 0 (offset 0)
Ignoring wrong pointing object 339 0 (offset 0)
Ignoring wrong pointing object 351 0 (offset 0)
Ignoring wrong pointing object 353 0 (offset 0)
Ignoring wrong pointing object 360 0 (offset 0)
Ignoring wrong pointing object 367 0 (offset 0)


59


# 4. Create embeddings for the document chunks and store them in vector DB

In [6]:
PERSIST_DIR = "vectorstore"
if os.path.exists(PERSIST_DIR):
    shutil.rmtree(PERSIST_DIR)

# this functions creates embeddings functions which can be used to generate embeddings


def embeddings_factory():
    return OpenAIEmbeddings(model="text-embedding-3-small", api_key=OPENAI_API_KEY)


embeddings_function = embeddings_factory()

run_embedding_test = False
# a simple test to perform embedding test to see how different words are closer to each other
if (run_embedding_test):
    from langchain.evaluation import load_evaluator
    evaluator = load_evaluator(
        evaluator="embedding_distance", embeddings=embeddings_function)
    print(evaluator._evaluate_strings(prediction="India", reference="Taj Mahal"))

# create a vector store and pass in embeddings_function as the helper function
vectorstore = Chroma.from_documents(
    documents=chunks, embedding=embeddings_function, persist_directory=PERSIST_DIR)
vectorstore.persist()

/var/folders/7m/bl62sr3d3m17_5tx_tly49tw0000gn/T/ipykernel_46380/1761815814.py:25: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


# 5. Query for relevant data from the vectorstore

In [7]:
# create a retriever based on similarity search which will search for the relevant docs based on the user query
retriever = vectorstore.as_retriever(search_type="similarity")


def search_vectorstore(user_query, retriever):
    docs = retriever.invoke(user_query)
    return docs


# following is the question asked by user
user_query = "Put the religion Islam, Chirstianity and Judaism in order of their origination with approx year"
searched_docs = search_vectorstore(user_query=user_query, retriever=retriever)


# define a function to concatenate the searched_docs for later use
def join_relevant_docs(docs):
    relevant_info = "\n\n---\n\n".join(
        each_doc.page_content for each_doc in docs)
    return relevant_info

# # 6. Create the prompt for LLM with the relevant info extracted from vectorstore

In [8]:
# create a prompt temaplate for the LLM to follow
prompt_template_str = """
You are an assistant for question-answering tasks. 
Strictly use the following pieces of retrieved context to answer the question. 
If you don't know the answer or do not find the relevant information in the context, just say that you don't know. 
Use three sentences maximum and keep the answer concise.

Question: {question} 

Context: {context} 

Answer:
"""

# create a prompt and pass it on to LLM
prompt_template = ChatPromptTemplate.from_template(prompt_template_str)
promt = prompt_template.format(
    context=join_relevant_docs(searched_docs), question=user_query)
print(promt)

Human: 
You are an assistant for question-answering tasks. 
Strictly use the following pieces of retrieved context to answer the question. 
If you don't know the answer or do not find the relevant information in the context, just say that you don't know. 
Use three sentences maximum and keep the answer concise.

Question: Put the religion Islam, Chirstianity and Judaism in order of their origination with approx year 

Context: Christianity, Islam, and Judaism are all monotheistic (believe in one god) 
religions. All three religions were broadly founded on the continent of Asia in 
the geographic region called the Middle East. These three faiths share 
common history and traditions. All three have a respect for the Bible, 
especially the Hebrew Bible or “Old Testament” as all three religions believe 
in many of the same prophets (messengers of God) such as Abraham and 
Moses. They all maintain that there is one God and that Jerusalem is a holy 
city. However, Christianity, Islam and Jud

# 7. Invoke LLM with the above generated prompt

In [9]:
response = llm.invoke(promt)
print(response.content)

Judaism originated around 1300 BCE, followed by Christianity in the 1st century CE, and then Islam in the 7th century CE.


# 8. Alternate Method of calling the llm and passing prompt using LangChain Chain

In [10]:
rag_chain = (
    {"context": retriever | join_relevant_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
)
AIMessage = rag_chain.invoke(user_query)
print(AIMessage.content)

Judaism originated around 1300 BCE, followed by Christianity in the 1st century CE, and then Islam in the 7th century CE.
